This The implementation of the S-DES algorithm:


1. P-BOX:


In [24]:
initial_p_box_mapping = [2,6,3,1,4,8,5,7]
final_p_box_mapping = [4,1,3,5,7,2,8,6]
compression_p_box_mapping = [6,3,7,4,8,5,10,9]
straight_p_box_mapping = [3,5,2,7,4,10,1,9,8,6]
straight_p_box_mapping_small = [2,4,3,1]
expansion_p_box_mapping = [4,1,2,3,2,3,4,1]
def generalBox(data,mapping):
    return [data[i - 1] for i in mapping]

2. Key generation:


In [25]:
def leftShift(data,numberOfShofts):
    for shifts in range(numberOfShofts):
        data.append(data.pop(0))
    return data

def keyGeneration(key):
    keys = []
    data = [*str(key)]
    data = generalBox(data,straight_p_box_mapping)
    
    k1 = data[:5]
    k2 = data[5:]
    
    k1 = leftShift(k1, 1)
    k2 = leftShift(k2, 1)
    
    keys.append(generalBox(k1+k2,compression_p_box_mapping))
    
    k1 = leftShift(k1, 2)
    k2 = leftShift(k2, 2)
    
    keys.append(generalBox(k1+k2,compression_p_box_mapping))
    
    return keys

3. The S-DES round:


In [26]:
one_s_box_mapping = [[1,0,3,2],[3,2,1,0],[0,2,1,3],[3,1,3,2]]
two_s_box_mapping = [[0,1,2,3],[2,0,1,3],[3,0,1,0],[2,1,0,3]]
def sBox_permutation(data,mapping):
    for block_num, block in enumerate([data]):
        i = int(block[0] + block[3], 2)
        j = int(''.join([block[x] for x in range(1, 3)]), 2)
        block = ("{0:b}".format(mapping[i][j])).zfill(4)
    return block[2:]

def function(r,key):
    r = generalBox([*str(r)],expansion_p_box_mapping)
    
    xor_result = int(r.join('')) ^ int(key.join(''))
    
    k1 = xor_result[:5]
    k2 = xor_result[5:]
    
    sBoxResult = sBox_permutation(k1,one_s_box_mapping) + sBox_permutation(k2,two_s_box_mapping)
    
    return generalBox([*str(sBoxResult)],straight_p_box_mapping)

def mixer(l,r,key):
    function_result = function(r, key)
    
    xor_result = int(l.join('')) ^ int(function_result.join(''))
    
    return [xor_result,r]

def Round(l,r,withSwapper):
    mixerResutl = mixer(l,r,'')
    if(withSwapper):
        return [mixerResutl[1],mixerResutl[0]]  
    else:
        return mixerResutl

4. Compine everything to encrpyt and decrypt: